In [1]:
#!pip install dash
#!pip install xgboost

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
import joblib  # cargar el modelo entrenado
import sklearn

In [3]:
print(f"versión sklearn {sklearn.__version__} \n" 
f"versión dash {dash.__version__}")

versión sklearn 1.5.2 
versión dash 2.18.2


In [4]:
app = dash.Dash()

In [5]:
# Cargar el modelo
modelo_diabetes = joblib.load('diabetes_NeuralNet.pkl')  

In [6]:
edad_min = 18
edad_max = 120
estatura_min = 1
estatura_max = 2.5
peso_min = 10
peso_max = 300


app.layout = html.Div([
    html.H1('Detección Riesgo de Diabetes', style={'textAlign': 'center', 'color': 'blue', 'fontSize': '36px',  'fontWeight': 'bold'}),
    html.H2('''Esta herramienta es una guía para indicar si el usuario está en riesgo de padecer o no diabetes 
    y en ningún caso reemplaza el diagnóstico médico.'''),
    
    html.Div([
        
        # Dropdown Estado General de Salud
        html.Label('En general, su estado de salud es:', style={'fontSize': 18, 'marginBottom': '20px', 'marginTop': '20px'}),  # Título
        dcc.Dropdown(
            id='GenHlth',
            options = [
                {'label':'Excelente', 'value':1}, {'label':'Muy bueno', 'value':2}, {'label':'Bueno', 'value':3},
                {'label':'Regular', 'value':4}, {'label':'Mala', 'value':5}
            ],
        style={'width': '300px', 'marginBottom': '20px', 'flex': 1} ), # Ancho del Dropdown 

        # Edad
        html.Label('Edad (18 a 120 años):', style={'fontSize': 18, 'marginBottom': '20px'}),
        dcc.Input(id='edad', type='number', step='any', placeholder='Ej: 37', min=edad_min, max=edad_max,
                  style={'width': '300px', 'marginBottom': '20px', 'marginRight': '20px', 'flex': 1}) 

        ]),


    
    html.Div([
        # Estatura
        html.Label('Estatura (en metros):', style={'fontSize': 18, 'marginBottom': '20px'}),
        dcc.Input(id='height', type='number', step='any', placeholder='Ej: 1.75', min=estatura_min, max=estatura_max, 
                  style={'width': '300px', 'marginBottom': '20px', 'marginRight': '20px'}),

        # Peso
        html.Label('Peso (en kg):', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.Input(id='weight', type='number', step='any', placeholder='Ej: 70', min=peso_min, max=peso_max,   
                  style={'width': '300px', 'marginBottom': '20px'}),
    
        # Mostrar el resultado del IMC
        html.Div(id='BMI', style={'fontSize': 25, 'marginBottom': '20px'}),
        ]),
    html.Div([
        # Presión Arterial Alta
        html.Label('¿Tiene presión arterial alta?:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HighBP',  
        options=[
            {'label': 'Si', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

        # Colesterol Alto
        html.Label('¿Tiene colesterol alto?:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HighChol',  
        options=[
            {'label': 'Si', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

        # Ataque Cardíaco
        html.Label('Enfermedad coronaria o ataque cardíaco:', style={'fontSize': 18, 'marginBottom': '10px'}),
        dcc.RadioItems(
        id='HeartDiseaseorAttack',  
        options=[
            {'label': 'Si', 'value': 1},  
            {'label': 'No', 'value': 0}  
        ],
        labelStyle={'display': 'block', 'marginBottom': '10px'} ),

         ]),

         # Contenedor para la predicción, centrado al final
        html.Div(id='prediccion_diabetes', style={'fontSize': 35, 'marginTop': '40px', 'textAlign': 'center', 'fontWeight': 'bold'}),

    # dcc.Store para almacenar variables:
    dcc.Store(id='store_age'), # rango de edad
    dcc.Store(id='store_imc') # IMC

        
])




# Calcular el rango de edad 
@app.callback(
    Output('store_age', 'data'),
    Input('edad', 'value')
)
def rango_edad(edad):
    if edad and (edad_min <= edad <= edad_max):
        try: 
            if 18 <= edad <= 24:
                Age = 1 
            elif edad <= 29:
                Age = 2 
            elif edad <= 34:
                Age = 3  
            elif edad <= 39:
                Age = 4  
            elif edad <= 44:
                Age = 5   
            elif edad <= 49:
                Age = 6  
            elif edad <= 54:
                Age = 7  
            elif edad <= 59:
                Age = 8  
            elif edad <= 64:
                Age = 9  
            elif edad <= 69:
                Age = 10  
            elif edad <= 74:
                Age = 11  
            elif edad <= 79:
                Age = 12  
            else:
                Age = 13 

            return {'age': Age}
        
        except ValueError:
            return {'age': None}
    return {'age': None}




# Callback para calcular el IMC/BMI
@app.callback(
    Output('store_imc', 'data'),
    Output('BMI', 'children'),
    Input('height', 'value'),
    Input('weight', 'value')
)
def calcular_imc(height, weight):
    if (height and weight):
        try:
            # Calcular el IMC
            BMI = float(weight) / (float(height) ** 2)
            
            if BMI < 18.5:
                bmi_message = f"Su IMC es {BMI:.0f}: Bajo peso"
                color = '#20c1c9'
            elif BMI <= 24.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Peso normal"
                color = '#20c975'
            elif BMI <= 29.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Sobrepeso"
                color = '#decf28'
            elif BMI <= 34.9:
                bmi_message = f"Su IMC es {BMI:.0f}: Obesidad"
                color = '#f5bf2c'
            else:
                bmi_message = f"Su IMC es {BMI:.0f}: Obesidad mórbida"
                color = '#bf2626'
        
            return {'imc': round(BMI)}, html.Div(bmi_message, style={'color': color, 'fontSize': '24px'})
        
        except ValueError:
            bmi_message = "Por favor valide los valores de estatura y peso ingresados."
            return {'imc': None}, html.Div(bmi_message, style={'color': 'red', 'fontSize': '24px'})

    bmi_message = "Ingrese su estatura y peso para calcular el IMC."
    return {'imc': None}, html.Div(bmi_message, style={'color': 'red', 'fontSize': '24px'})


@app.callback(
    Output('prediccion_diabetes', 'children'),
    Input('HighBP', 'value'),
    Input('HighChol', 'value'),
    Input('HeartDiseaseorAttack', 'value'),
    Input('GenHlth', 'value'),
    Input('store_age', 'data'),
    Input('store_imc', 'data')
)
def calcular_imc_and_predict(HighBP, HighChol, HeartDiseaseorAttack, GenHlth, store_age, store_imc):
    Age = store_age.get('age', None)
    BMI = store_imc.get('imc', None)
    
    if HighBP is not None and HighChol is not None and HeartDiseaseorAttack is not None and GenHlth is not None and Age is not None and BMI is not None:

        
        # Preparar los datos para la predicción - orden dado por el entrenamiento del modelo
        feature_names = ["HighBP", "HighChol", "BMI", "HeartDiseaseorAttack", "GenHlth", "Age"] 
        input_data = pd.DataFrame([[HighBP, HighChol, BMI,  HeartDiseaseorAttack, GenHlth, Age]], columns=feature_names)

        # Usar el modelo para hacer la predicción
        try:
            prediccion_diabetes = modelo_diabetes.predict(input_data)  # Usar el modelo cargado
            if (prediccion_diabetes) == 1:
                prediccion_message = f"Alto riesgo de diabetes."
                color1 = 'red'
            else:
                prediccion_message = f"Bajo riesgo de diabetes."
                color1 = 'green'
        except Exception as e:
            prediccion_message = f"Error en la predicción: {str(e)}"
        
        return html.Div(prediccion_message, style={'color': color1, 'fontSize': '35px'})

    return "Por favor, ingrese todos los datos solicitados"


In [7]:
if __name__ == '__main__':
    app.run_server()

In [8]:
# Verificación manual de las predicciones del modelo

import pandas as pd
import numpy as np

# nombre de los features del modelo - tener en cuenta el orden en que se hizo el training del modelo
feature_names = ["HighBP", "HighChol", "BMI", "HeartDiseaseorAttack", "GenHlth", "Age"]

# Input data
HighBP = 1
HighChol = 1
BMI = 29
HeartDiseaseorAttack = 0
GenHlth = 2
Age = 7  # rango de la edad 

# convertir los datos a un DataFrame
input_data = pd.DataFrame([[HighBP, HighChol, BMI,  HeartDiseaseorAttack, GenHlth, Age]], columns=feature_names)

# Realizar la predicción 
prediccion_diabetes = modelo_diabetes.predict(input_data)

# Imprimir la predicción
(prediccion_diabetes)

array([1.])

In [9]:
# Imprimir el DF enviado para la predicción
input_data

,HighBP,HighChol,BMI,HeartDiseaseorAttack,GenHlth,Age
0,1,1,29,0,2,7
